# Определение стоимости автомобилей

**Описание проекта:**
     
Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей.

Заказчику важны:

- качество предсказания
- скорость предсказания
- время обучения

**Цель исследования:**  
Построить модель для определения стоимости автомобилей с наилучшим качеством предсказания, минимальной скоростью обучения и предсказания.

**Ход исследования:**  
* Изучение данных
* Предобработка данных
* Подготовка данных к обучению моделей
* Обучение моделей
* Анализ скорости обучения и предсказания, качества предсказания моделей
* Выбор лучшей модели и проверка её на тестовой выборке

## Подготовка данных

In [1]:
#Импортируем библиотеки
from time import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
#Сохраним данные в переменную data
data = pd.read_csv('/datasets/autos.csv')

Изучим данные

In [3]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>

RangeIndex: 354369 entries, 0 to 354368

Data columns (total 16 columns):

 #   Column             Non-Null Count   Dtype 

---  ------             --------------   ----- 

 0   DateCrawled        354369 non-null  object

 1   Price              354369 non-null  int64 

 2   VehicleType        316879 non-null  object

 3   RegistrationYear   354369 non-null  int64 

 4   Gearbox            334536 non-null  object

 5   Power              354369 non-null  int64 

 6   Model              334664 non-null  object

 7   Kilometer          354369 non-null  int64 

 8   RegistrationMonth  354369 non-null  int64 

 9   FuelType           321474 non-null  object

 10  Brand              354369 non-null  object

 11  Repaired           283215 non-null  object

 12  DateCreated        354369 non-null  object

 13  NumberOfPictures   354369 non-null  int64 

 14  PostalCode         354369 non-null  int64 

 15  LastSeen           354369 non-null  object

dtyp

In [5]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [6]:
#Приведём названия столбцов к нижнему регистру
data.columns = data.columns.str.lower()

In [7]:
#Приведём названия к змеиному регистру
data = data.rename(columns={'datecrawled':'date_crawled', 
                        'vehicletype':'vehicle_type', 
                        'registrationyear':'registration_year', 
                        'registrationmonth':'registration_month', 
                        'fueltype':'fuel_type', 
                        'repaired':'repaired', 
                        'datecreated':'date_created',
                        'numberofpictures': 'number_of_pictures',
                        'postalcode':'postal_code',
                        'lastseen':'last_seen'})

In [8]:
#Удалим неинформативные столбцы
data = data.drop(['number_of_pictures', 'postal_code', 'date_created', 'last_seen', 'registration_month', 'date_crawled'], axis=1)

In [9]:
#Узнаем количество пропусков
data.isna().sum()

price                    0
vehicle_type         37490
registration_year        0
gearbox              19833
power                    0
model                19705
kilometer                0
fuel_type            32895
brand                    0
repaired             71154
dtype: int64

In [10]:
#Заменим пропуски значением unknown
data['vehicle_type'] = data['vehicle_type'].fillna('unknown')
data['gearbox'] = data['gearbox'].fillna('unknown')
data['model'] = data['model'].fillna('unknown')
data['fuel_type'] = data['fuel_type'].fillna('unknown')
data['repaired'] = data['repaired'].fillna('unknown')

In [11]:
#Проверим количество пропусков после замены
data.isna().sum()

price                0
vehicle_type         0
registration_year    0
gearbox              0
power                0
model                0
kilometer            0
fuel_type            0
brand                0
repaired             0
dtype: int64

In [12]:
#Проверим наличие дубликатов
data.duplicated().sum()

45040

In [13]:
#Удалим дубликаты
data = data.drop_duplicates()

In [14]:
#Проверим наличие дубликатов после удаления
data.duplicated().sum()

0

In [15]:
#Найдём аномалии в столбцах
#Выведем уникальные значения столбцов
data['price'].value_counts().sort_index()

0        8904
1        1075
2          12
3           7
4           1
         ... 
19995       9
19997       1
19998       5
19999     252
20000     223
Name: price, Length: 3731, dtype: int64

In [16]:
#Удалим строки со значением 0
data = data[data['price']>=1]

In [17]:
data['vehicle_type'].value_counts().sort_index()

bus            25455
convertible    17893
coupe          14234
other           2975
sedan          76513
small          66008
suv            10596
unknown        31670
wagon          55081
Name: vehicle_type, dtype: int64

**В столбце vehicle_type аномалий нет**

In [18]:
data['registration_year'].value_counts().sort_index()

1000    30
1001     1
1039     1
1111     1
1234     4
        ..
8500     1
8888     1
9000     2
9450     1
9999    18
Name: registration_year, Length: 140, dtype: int64

In [19]:
#Удалим строки созначением меньше 1950 и больше 2023
data = data.loc[(data['registration_year'] <= 2023) & (data['registration_year'] >= 1950)]

In [20]:
data['gearbox'].value_counts().sort_index()

auto        58083
manual     226829
unknown     15258
Name: gearbox, dtype: int64

**В столбце gearbox аномалий нет**

In [21]:
data['power'].value_counts().sort_index()

0        32799
1           25
2            9
3            8
4           26
         ...  
17932        1
19208        1
19211        1
19312        1
20000        1
Name: power, Length: 699, dtype: int64

In [22]:
#Удалим строки со значением меньше 1 и больше 700
data = data.loc[(data['power'] >= 1) & (data['power'] <= 700)]

In [23]:
data['model'].value_counts().sort_index()

100         330
145          36
147         480
156         500
159         189
           ... 
yaris       838
yeti        158
ypsilon     161
z_reihe     637
zafira     2246
Name: model, Length: 250, dtype: int64

**В столбце model аномалий нет**

In [24]:
data['kilometer'].value_counts().sort_index()

5000        2960
10000        892
20000       3135
30000       3606
40000       4140
50000       5283
60000       6224
70000       7359
80000       8432
90000       9599
100000     11948
125000     29522
150000    173954
Name: kilometer, dtype: int64

**В столбце kilometer аномалий нет**

In [25]:
data['fuel_type'].value_counts().sort_index()

cng            459
electric        75
gasoline     77297
hybrid         195
lpg           4397
other           92
petrol      165514
unknown      19025
Name: fuel_type, dtype: int64

**В столбце fuel_type аномалий нет**

In [26]:
data['brand'].value_counts().sort_index()

alfa_romeo         1882
audi              22973
bmw               28352
chevrolet          1408
chrysler           1121
citroen            4096
dacia               754
daewoo              398
daihatsu            587
fiat               7121
ford              18915
honda              2298
hyundai            2926
jaguar              419
jeep                558
kia                1971
lada                148
lancia              373
land_rover          434
mazda              4424
mercedes_benz     25001
mini               2724
mitsubishi         2405
nissan             3766
opel              28608
peugeot            8562
porsche             583
renault           12740
rover               355
saab                445
seat               5480
skoda              4562
smart              4218
sonstige_autos     1795
subaru              628
suzuki             1863
toyota             3789
trabant             284
volkswagen        55502
volvo              2586
Name: brand, dtype: int64

**В столбце brand аномалий нет**

In [27]:
data['repaired'].value_counts().sort_index()

no         196063
unknown     43506
yes         27485
Name: repaired, dtype: int64

**В столбце repaired аномалий нет**

**Вывод:**  
* Данные проанализированы
* Удалены дубликаты
* Заполнены пропуски
* Удалены неинформативные столбцы, которые не понадобятся для обучения моделей
* Оработаны аномалии в столбцах

## Обучение моделей

### Подготовка к обучению

In [28]:
#Переведём категориальные значения в количественные с помощью ОНЕ
data_ohe = data.copy()
data_ohe = pd.get_dummies(data, drop_first=True)
data_ohe.head()

,price,registration_year,power,kilometer,vehicle_type_convertible,vehicle_type_coupe,vehicle_type_other,vehicle_type_sedan,vehicle_type_small,vehicle_type_suv,...,brand_smart,brand_sonstige_autos,brand_subaru,brand_suzuki,brand_toyota,brand_trabant,brand_volkswagen,brand_volvo,repaired_unknown,repaired_yes
1,18300,2011,190,125000,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,9800,2004,163,125000,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1500,2001,75,150000,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
4,3600,2008,69,90000,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,650,1995,102,150000,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [29]:
#Разделим данные на обучающую и тестовую выборки
target_ohe = data_ohe['price']
features_ohe = data_ohe.drop('price', axis=1)
#features_train_ohe, target_train_ohe = train_test_split(features_ohe, target_ohe, test_size=0.25, random_state=12345) 

features_train_ohe, x_ohe, target_train_ohe, y_ohe = train_test_split(features_ohe, target_ohe, test_size=0.4, random_state=12345)
features_valid_ohe, features_test_ohe, target_valid_ohe, target_test_ohe = train_test_split(x_ohe, y_ohe, test_size=0.5, random_state=12345)

print(features_train_ohe.shape)
print(features_test_ohe.shape)
print(features_valid_ohe.shape)

(160232, 310)

(53411, 310)

(53411, 310)


In [30]:
#Проведём масштабирование признаков
scaler = StandardScaler()
scaler.fit(features_train_ohe)
features_train_ohe = scaler.transform(features_train_ohe)
features_test_ohe = scaler.transform(features_test_ohe)
features_valid_ohe = scaler.transform(features_valid_ohe)

### Линейная регрессия:

In [31]:
#Обучим модель линейной регрессии, сделаем предсказания и найдем RMSE

In [32]:
%%time
model_lr = LinearRegression()
model_lr.fit(features_train_ohe, target_train_ohe)
predictions_valid_lr = model_lr.predict(features_valid_ohe)
rmse_lr = mean_squared_error(target_valid_ohe, predictions_valid_lr)**0.5
print(rmse_lr)

2653.4809371960673

CPU times: user 17.3 s, sys: 19.5 s, total: 36.9 s

Wall time: 36.8 s


Значение RMSE выше допустимого, обучим другие модели

### Случайный лес

In [33]:
#Проведём кодирование категориальных признаков с помощью OrdinalEncoder
data_enc = data.copy()
cod_feat = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'repaired']
enc = OrdinalEncoder()
enc.fit(data_enc[cod_feat])
data_enc[cod_feat] = enc.transform(data_enc[cod_feat])

In [34]:
#Разделим данные на обучающую и тестовую выборки
target_enc = data_enc['price']
features_enc = data_enc.drop('price', axis=1)
#features_train_enc, features_test_enc, target_train_enc, target_test_enc = train_test_split(features_enc, target_enc, test_size=0.25, random_state=12345) 

features_train_enc, x_enc, target_train_enc, y_enc = train_test_split(features_enc, target_enc, test_size=0.4, random_state=12345)
features_valid_enc, features_test_enc, target_valid_enc, target_test_enc = train_test_split(x_enc, y_enc, test_size=0.5, random_state=12345)

print(features_train_enc.shape)
print(features_test_enc.shape)
print(features_valid_enc.shape)

(160232, 9)

(53411, 9)

(53411, 9)


In [35]:
#Обучим модель случайного леса

In [36]:
%%time
for est in [30, 60, 100]:
    model_forest = RandomForestRegressor(random_state=12345, 
                                   n_estimators=est, 
                                   max_depth=10)
    model_forest.fit(features_train_enc, target_train_enc)
    predictions_valid_enc = model_forest.predict(features_valid_enc)
    rmse_forest = mean_squared_error(target_valid_enc, predictions_valid_enc)**0.5
    print('Количество деревьев:', est)
    print('RMSE случайного леса:', rmse_forest)
    print('')
print()

Количество деревьев: 30

RMSE случайного леса: 1935.610283949783



Количество деревьев: 60

RMSE случайного леса: 1932.2299961294684



Количество деревьев: 100

RMSE случайного леса: 1929.399373867918





CPU times: user 40.4 s, sys: 0 ns, total: 40.4 s

Wall time: 40.4 s


Лучший результат RMSE у леса с количеством деревьев = 100

### LightGBM

In [37]:
!pip install lightgbm

In [38]:
#Обучим модель без подбора гиперпараметров

In [39]:
%%time
model_lg = lgb.LGBMRegressor(random_state=12345)
model_lg.fit(features_train_enc, target_train_enc)
predictions_valid_lg = model_lg.predict(features_valid_enc)
rmse_lg = mean_squared_error(target_valid_enc, predictions_valid_lg)**0.5
print(rmse_lg)

1746.7440409918822

CPU times: user 5min 41s, sys: 6.45 s, total: 5min 48s

Wall time: 5min 50s


In [ ]:
%%time
params_gr = {'n_estimators': [30, 60, 100], 'num_leaves': [50, 70, 100], 'max_depth': [10, 12]}

grid = GridSearchCV(model_lg, params_gr, scoring='neg_root_mean_squared_error', cv=5)

grid.fit(features_train_enc, target_train_enc)

print(grid.best_params_)
print(grid.best_score_*-1)

Лучший результат получен при параметрах: **max_depth** = 12, **n_estimators** = 100, **num_leaves** = 100

## Анализ моделей

In [41]:
#Проверим время обучения линейной регрессии

In [55]:
%%time
model_lr = LinearRegression()
model_lr.fit(features_train_ohe, target_train_ohe)

CPU times: user 13.7 s, sys: 10.8 s, total: 24.5 s

Wall time: 24.5 s


LinearRegression()

In [43]:
#Проверим скорость предсказания линейной регрессии

In [44]:
%%time
predictions_valid_lr = model_lr.predict(features_valid_ohe)

CPU times: user 61 ms, sys: 31.4 ms, total: 92.4 ms

Wall time: 81.3 ms


In [45]:
#Проверим время обучения случайного леса

In [ ]:
%%time
model_forest = RandomForestRegressor(random_state=12345, n_estimators=100, max_depth=10)
model_forest.fit(features_train_enc, target_train_enc)

In [47]:
#Проверим скорость предсказания случайного леса

In [48]:
%%time
predictions_valid_enc = model_forest.predict(features_valid_enc)

CPU times: user 460 ms, sys: 77 µs, total: 460 ms

Wall time: 460 ms


In [49]:
#Проверим время обучения LightGBM

In [50]:
%%time
model_lg = lgb.LGBMRegressor(random_state=12345, max_depth=12, n_estimators=100)
model_lg.fit(features_train_enc, target_train_enc)

CPU times: user 37 s, sys: 667 ms, total: 37.7 s

Wall time: 37.9 s


LGBMRegressor(max_depth=12, random_state=12345)

In [51]:
#Проверим скорость предсказания LightGBM

In [52]:
%%time
predictions_valid_lg = model_lg.predict(features_valid_enc)

CPU times: user 403 ms, sys: 0 ns, total: 403 ms

Wall time: 389 ms


Быстрее всего обучается модель LightGBM, также у неё самое меньшее значение RMSE. Проверим её на тестовой выборке.

In [53]:
%%time
model_lg = lgb.LGBMRegressor(random_state=12345, max_depth=12, n_estimators=100)
model_lg.fit(features_train_enc, target_train_enc)
predictions_test_lg = model_lg.predict(features_test_enc)
rmse_lg = mean_squared_error(target_test_enc, predictions_test_lg)**0.5
print(rmse_lg)

1726.2095467569854

CPU times: user 43.6 s, sys: 680 ms, total: 44.3 s

Wall time: 44.6 s


Модель хорошо показала себя на тестовой выборке. Время обучения и предсказания небольшое. RMSE в пределах допустимого. Рекомендуем использовать модель LightGBM для определения стоимости автомобилей.

**Вывод:**  
* На вход были получены сырые данные. Мы их привели в нормальный вид для дальнейшей работы
* Подготовили данные для обучения моделей
    + Масштабировали признаки
    + Провели кодировку признаков
* Выбрали для обучения три модели: Линейную регрессию, случайный лес и LightGBM, и рассчитали для каждой RMSE
* Обучили модели и провели анализ резельтатов обучения
* По результатам исследования была выбрана модель LightGBM. Исходя из указанных критериев, она подходит лучше всего для определения стоимости автомобилей